In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import problem
import warnings
import glob
import re
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [101]:
list_files = glob.glob("./data/data_passengers/*.txt")

In [102]:
origin_files = []
destin_files = []

for i in list_files :
    type_file = re.findall(r"(?=_)*[a-z]+(?=\.)",i)[0]
    if type_file == 'origin':
        origin_files.append(i)
    else : 
        destin_files.append(i)

In [103]:
def parse_data(file):
    
    IATA = re.findall(r"(?=/)*[A-Z]+",file)[0]
    where = re.findall(r"(?=_)*[a-z]+(?=\.)",file)[0]
    
    data = pd.read_table(file, delim_whitespace=True)
    data["IATA"] = IATA
    
    data = data[(data.Month.str.isnumeric()) & (
     (data.Year == 2011) |
     (data.Year == 2012) | 
     (data.Year == 2013))].drop(columns=["DOM","INTER"]) 
    
    
    data.TOTAL = data.TOTAL.apply(lambda x : int(x.replace(",",'')))
    data["Month"] = data.Month.astype(int)
    
    
    if where == "origin":
        data = data.rename(columns={"TOTAL":'TOTAL_DEPARTURE'})
    else :
        data = data.rename(columns={"TOTAL":'TOTAL_DESTINATION'})

    return data

In [104]:
origin_data = pd.concat([parse_data(i) for i in origin_files])
destin_data = pd.concat([parse_data(i) for i in destin_files])

In [105]:
Data_passengers = pd.merge(origin_data,destin_data,
        how='left',on=["Year",'Month','IATA'])

In [106]:
Data_passengers.head()

,Year,Month,TOTAL_DEPARTURE,IATA,TOTAL_DESTINATION
0,2011,1,1545190,MIA,1469003
1,2011,2,1333668,MIA,1336537
2,2011,3,1547992,MIA,1540702
3,2011,4,1483476,MIA,1479126
4,2011,5,1516452,MIA,1455400


In [107]:
external_data = pd.read_csv('submissions/submit/external_data.csv', 
                            index_col=0)
external_data["year"] = pd.to_datetime(external_data["Date"]).dt.year
external_data["month"] = pd.to_datetime(external_data["Date"]).dt.month

In [109]:
external_data = external_data.merge(Data_passengers, 
                   how='left',
                   left_on=["year",'month',"AirPort"],
                   right_on= ["Year",'Month',"IATA"]
                   ).drop(columns=["Year",'Month',"IATA"])

In [111]:
external_data.head(2)

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Holiday,year,month,TOTAL_DEPARTURE,TOTAL_DESTINATION
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,0,2011,9,3582088,3580851
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,0,2011,9,3582088,3580851


In [112]:
external_data.to_csv('submissions/submit/external_data.csv')